In [1]:
import numpy as np
import tensorflow as tf
import h5py
import random
import time
slim=tf.contrib.slim

In [2]:
f=h5py.File('data/fitt.h5','r')
data_x=f['x']
data_y=f['y']
data_x[:2]

array([[18.08,  2.49],
       [18.14,  2.48]])

# 数据准备

In [3]:
#读取数据
f=h5py.File('data/fitt.h5','r')
data_x=f['x']
data_y=f['y']


#分配训练测试集
num=list(range(data_x.shape[0]))
num1=random.sample(num,1000)
num2=set(num)-set(num1)
num2=list(num2)
num1.sort()


x_train=data_x[num2]
y_train=data_y[num2]

x_test=data_x[num1]
y_test=data_y[num1]
#生成批次
train_queue = tf.train.slice_input_producer([x_train,y_train],shuffle=None)
val_queue = tf.train.slice_input_producer([x_test,y_test],shuffle=None)
batch_xt,batch_yt=tf.train.shuffle_batch(train_queue,batch_size=16,capacity=500,min_after_dequeue=150)
batch_xv,batch_yv=tf.train.shuffle_batch(val_queue,batch_size=16,capacity=500,min_after_dequeue=150)

# 搭建模型

### 超参数设置

In [4]:
# Optmizer: 
learning_rate = 0.007  # Small lr helps not to diverge during training. 
nb_iters = 10000  # How many times we perform a training step (therefore how many times we show a batch). 
lr_decay = 0.92  # default: 0.9 . Simulated annealing.
momentum = 0.5  # default: 0.0 . Momentum technique in weights update
lambda_l2_reg = 0.003  # L2 regularization of weights - avoids overfitting

### 搭建神经网络

In [5]:
x=tf.placeholder(tf.float32,[None,2])
y=tf.placeholder(tf.float32,[None,1])
with slim.arg_scope([slim.layers.fully_connected],
                    weights_initializer=slim.xavier_initializer(),
                    activation_fn=tf.nn.relu):
    net=slim.fully_connected(x,4,scope='fc1')
#     net=slim.fully_connected(net,8,scope='fc2')
    net=slim.fully_connected(net,4,scope='fc3')
    output=slim.fully_connected(net,1,activation_fn=None,scope='fc4')

### 搭建损失函数 

In [6]:
# Training loss and optimizer

with tf.variable_scope('Loss'):
    # L2 loss
   
    output_loss = tf.reduce_mean(tf.nn.l2_loss(output - y))
        
    # L2 regularization (to avoid overfitting and to have a  better generalization capacity)
    reg_loss = 0
    for tf_var in tf.trainable_variables():
        if not ("Bias" in tf_var.name or "Output_" in tf_var.name):
            reg_loss += tf.reduce_mean(tf.nn.l2_loss(tf_var))
            
    loss = output_loss + lambda_l2_reg * reg_loss
    tf.summary.scalar('loss',loss)

with tf.variable_scope('acc'):
    acc = tf.reduce_mean(tf.nn.l2_loss(y-output))/tf.reduce_mean(tf.nn.l2_loss(y-0))
    
    acc=1-acc
    tf.summary.scalar('acc',acc)

with tf.variable_scope('Optimizer'):
    optimizer = tf.train.RMSPropOptimizer(learning_rate, decay=lr_decay, momentum=momentum)
    train_op = optimizer.minimize(loss)
    merged_summary = tf.summary.merge_all()

### 训练模型

In [7]:
min_loss=10000
sess=tf.Session()
# train_writer=tf.summary.FileWriter('C:/graph/fitted/train/',sess.graph)
# test_writer = tf.summary.FileWriter('C:/graph/fitted/test/', sess.graph)
# saver=tf.train.Saver(max_to_keep=3)

init_op = tf.global_variables_initializer()
sess.run(init_op)
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess, coord)
t1=time.time()
for training_itr in range(50000):
    x1, y1 = sess.run([batch_xt,batch_yt])
    feed_dict1 = {x:x1,y:y1}
    _, loss1,acc1,summaries1 = sess.run([train_op, loss,acc,merged_summary], feed_dict1)

#     train_writer.add_summary(summaries1, training_itr)
    if training_itr %5==0:
#             saver.save(sess=sess, save_path='model/hand_landmark_v6.1_model/model.ckpt',global_step=(global_step + 1))
        mean_val_loss = 0

        x2,y2=sess.run([batch_xv,batch_yv])
        feed_dict2 = {x:x2,y:y2}
        tt=time.time()
        loss2,acc2,summaries2 = sess.run([loss,acc,merged_summary], feed_dict2)

        print('step: {} , time : {}'.format(training_itr,time.time()-tt))
        print('train: loss: {} acc: {}  val: loss: {} acc: {}'.format(loss1,acc1,loss2,acc2))
#         test_writer.add_summary(summaries2, training_itr)
#         if loss1 < min_loss:
#             min_loss=loss1
#             saver.save(sess=sess, save_path='D:/model/fitted/model.ckpt',global_step=(training_itr + 1))
print('final time ： {}'.format(time.time()-t1))
sess.close()
coord.request_stop()
    






step: 0 , time : 0.018008947372436523
train: loss: 2398496.0 acc: -0.021650314331054688  val: loss: 11493932.0 acc: -0.008323311805725098
step: 5 , time : 0.0
train: loss: 5536794.0 acc: 0.0029897093772888184  val: loss: 10574709.0 acc: 0.002350449562072754
step: 10 , time : 0.0
train: loss: 4038916.25 acc: 0.019552290439605713  val: loss: 6597804.0 acc: 0.025935590267181396
step: 15 , time : 0.0010018348693847656
train: loss: 5282892.5 acc: 0.05013144016265869  val: loss: 6773632.0 acc: 0.06888723373413086
step: 20 , time : 0.0
train: loss: 5894154.0 acc: 0.10273385047912598  val: loss: 1518776.625 acc: 0.24219155311584473
step: 25 , time : 0.0
train: loss: 4544944.0 acc: 0.16228020191192627  val: loss: 7731868.5 acc: 0.1614583134651184
step: 30 , time : 0.0010018348693847656
train: loss: 3159933.0 acc: 0.2504119277000427  val: loss: 5395313.0 acc: 0.31889504194259644
step: 35 , time : 0.0
train: loss: 3243807.25 acc: 0.3505120277404785  val: loss: 3445324.25 acc: 0.4094967246055603
s

step: 405 , time : 0.0
train: loss: 481424.3125 acc: 0.9223803281784058  val: loss: 290830.8125 acc: 0.9559406042098999
step: 410 , time : 0.0
train: loss: 833183.5 acc: 0.9446377754211426  val: loss: 521007.34375 acc: 0.7078371644020081
step: 415 , time : 0.0010008811950683594
train: loss: 564037.875 acc: 0.9538982510566711  val: loss: 653002.0625 acc: 0.6656532287597656
step: 420 , time : 0.0
train: loss: 181913.453125 acc: 0.9517530798912048  val: loss: 1240253.25 acc: 0.6879330277442932
step: 425 , time : 0.001001119613647461
train: loss: 713342.5625 acc: 0.9771745800971985  val: loss: 848204.875 acc: 0.9100331664085388
step: 430 , time : 0.0
train: loss: 3227378.25 acc: 0.8707790374755859  val: loss: 195611.359375 acc: 0.9346972107887268
step: 435 , time : 0.0
train: loss: 1990656.875 acc: 0.932315468788147  val: loss: 1131318.625 acc: 0.7775947451591492
step: 440 , time : 0.0010004043579101562
train: loss: 2404147.5 acc: 0.8410935401916504  val: loss: 133235.4375 acc: 0.973995149

step: 810 , time : 0.0010006427764892578
train: loss: 784286.75 acc: 0.9434651136398315  val: loss: 430360.40625 acc: 0.9539437294006348
step: 815 , time : 0.0
train: loss: 869721.9375 acc: 0.927031934261322  val: loss: 192146.578125 acc: 0.9507090449333191
step: 820 , time : 0.0
train: loss: 223473.421875 acc: 0.9419749975204468  val: loss: 355806.90625 acc: 0.9373776912689209
step: 825 , time : 0.0
train: loss: 509070.875 acc: 0.908772885799408  val: loss: 1753887.375 acc: 0.7117499113082886
step: 830 , time : 0.0010006427764892578
train: loss: 951728.0625 acc: 0.8937259912490845  val: loss: 698718.875 acc: 0.8620364665985107
step: 835 , time : 0.0
train: loss: 789149.8125 acc: 0.8852525353431702  val: loss: 734750.8125 acc: 0.8397188186645508
step: 840 , time : 0.0010008811950683594
train: loss: 616630.8125 acc: 0.8101705312728882  val: loss: 412855.96875 acc: 0.8440320491790771
step: 845 , time : 0.0010006427764892578
train: loss: 542118.625 acc: 0.6531134247779846  val: loss: 4385

step: 1145 , time : 0.0
train: loss: 937387.1875 acc: 0.942436933517456  val: loss: 383144.125 acc: 0.9378708600997925
step: 1150 , time : 0.0010006427764892578
train: loss: 262225.71875 acc: 0.9491804838180542  val: loss: 587671.0 acc: 0.8488461375236511
step: 1155 , time : 0.001001119613647461
train: loss: 1520521.75 acc: 0.9003580808639526  val: loss: 324197.125 acc: 0.9481358528137207
step: 1160 , time : 0.0
train: loss: 977008.625 acc: 0.9581920504570007  val: loss: 1489318.75 acc: 0.8416479229927063
step: 1165 , time : 0.0
train: loss: 1779586.125 acc: 0.9330569505691528  val: loss: 889454.1875 acc: 0.9067158102989197
step: 1170 , time : 0.0010004043579101562
train: loss: 1797355.375 acc: 0.8919078707695007  val: loss: 612997.0 acc: 0.927200973033905
step: 1175 , time : 0.0
train: loss: 382033.25 acc: 0.9756484627723694  val: loss: 513372.5 acc: 0.9387797713279724
step: 1180 , time : 0.0
train: loss: 600847.375 acc: 0.92554771900177  val: loss: 504255.6875 acc: 0.6989320516586304

step: 1490 , time : 0.0
train: loss: 417969.3125 acc: 0.9516592621803284  val: loss: 2906035.25 acc: 0.5900057554244995
step: 1495 , time : 0.0
train: loss: 103317.046875 acc: 0.9865365624427795  val: loss: 811593.875 acc: 0.9027464985847473
step: 1500 , time : 0.0
train: loss: 256214.875 acc: 0.9706386923789978  val: loss: 501702.46875 acc: 0.9467088580131531
step: 1505 , time : 0.0
train: loss: 300878.40625 acc: 0.9772555828094482  val: loss: 715607.4375 acc: 0.9494674205780029
step: 1510 , time : 0.0010008811950683594
train: loss: 473288.46875 acc: 0.9693734645843506  val: loss: 1033060.0625 acc: 0.5786436200141907
step: 1515 , time : 0.0010008811950683594
train: loss: 281598.21875 acc: 0.9484043121337891  val: loss: 1269889.0 acc: 0.40341025590896606
step: 1520 , time : 0.0010008811950683594
train: loss: 1750671.375 acc: 0.8074809312820435  val: loss: 285556.34375 acc: 0.9716757535934448
step: 1525 , time : 0.0
train: loss: 703452.3125 acc: 0.9690589904785156  val: loss: 860441.187

...

step: 48460 , time : 0.0010013580322265625
train: loss: 89176.96875 acc: 0.9005239009857178  val: loss: 214059.9375 acc: 0.8359405398368835
step: 48465 , time : 0.0
train: loss: 161026.90625 acc: 0.8861470222473145  val: loss: 4949080.0 acc: 0.518706202507019
step: 48470 , time : 0.0
train: loss: 425743.0625 acc: 0.7942018508911133  val: loss: 2306133.25 acc: 0.6503201127052307
step: 48475 , time : 0.0010006427764892578
train: loss: 195305.640625 acc: 0.8114562630653381  val: loss: 1437473.375 acc: 0.7249886989593506
step: 48480 , time : 0.0010006427764892578
train: loss: 142072.546875 acc: 0.8492273688316345  val: loss: 1068285.75 acc: 0.7758457064628601
step: 48485 , time : 0.001001119613647461
train: loss: 663848.125 acc: 0.766127347946167  val: loss: 1730429.25 acc: 0.7091784477233887
step: 48490 , time : 0.0
train: loss: 1444773.25 acc: 0.7660607099533081  val: loss: 700397.1875 acc: 0.7885182499885559
step: 48495 , time : 0.015625476837158203
train: loss: 928542.0625 acc: 0.88903

step: 48795 , time : 0.0
train: loss: 115454.53125 acc: 0.9315481781959534  val: loss: 2867130.0 acc: 0.6859367489814758
step: 48800 , time : 0.001001119613647461
train: loss: 174933.609375 acc: 0.886979341506958  val: loss: 1302677.75 acc: 0.6883336901664734
step: 48805 , time : 0.0
train: loss: 97910.6796875 acc: 0.9334127306938171  val: loss: 3491488.0 acc: 0.5527271032333374
step: 48810 , time : 0.0
train: loss: 64234.33984375 acc: 0.9264572858810425  val: loss: 3828961.25 acc: 0.517196774482727
step: 48815 , time : 0.0
train: loss: 9838.75 acc: 0.9870732426643372  val: loss: 2584950.75 acc: 0.7168159484863281
step: 48820 , time : 0.0
train: loss: 50869.5703125 acc: 0.9420531392097473  val: loss: 550560.625 acc: 0.7732546925544739
step: 48825 , time : 0.0
train: loss: 154154.671875 acc: 0.901404857635498  val: loss: 2242758.25 acc: 0.6716887950897217
step: 48830 , time : 0.0
train: loss: 136058.0 acc: 0.9011818170547485  val: loss: 813842.25 acc: 0.7750858068466187
step: 48835 , ti

step: 49130 , time : 0.0
train: loss: 832921.25 acc: 0.6239160299301147  val: loss: 871091.0 acc: 0.7507984638214111
step: 49135 , time : 0.0010006427764892578
train: loss: 235819.0 acc: 0.8288556337356567  val: loss: 2058228.875 acc: 0.7073975801467896
step: 49140 , time : 0.0010008811950683594
train: loss: 230080.390625 acc: 0.8657752275466919  val: loss: 1942027.0 acc: 0.6670013666152954
step: 49145 , time : 0.0
train: loss: 364694.0 acc: 0.8163894414901733  val: loss: 1232840.375 acc: 0.6945863962173462
step: 49150 , time : 0.0
train: loss: 192436.9375 acc: 0.8773866891860962  val: loss: 1415935.25 acc: 0.6497139930725098
step: 49155 , time : 0.0
train: loss: 497159.15625 acc: 0.7513449788093567  val: loss: 6079247.5 acc: 0.4241069555282593
step: 49160 , time : 0.0
train: loss: 58584.20703125 acc: 0.9589979648590088  val: loss: 3104699.75 acc: 0.5027216076850891
step: 49165 , time : 0.0
train: loss: 289731.96875 acc: 0.8625164031982422  val: loss: 1174141.375 acc: 0.669249176979064

step: 49465 , time : 0.0
train: loss: 193048.875 acc: 0.9827277660369873  val: loss: 846328.25 acc: 0.8186891674995422
step: 49470 , time : 0.0010008811950683594
train: loss: 1819653.125 acc: 0.37983548641204834  val: loss: 1002671.625 acc: 0.763115644454956
step: 49475 , time : 0.0010008811950683594
train: loss: 2732268.25 acc: -0.9515210390090942  val: loss: 1224543.75 acc: 0.6913001537322998
step: 49480 , time : 0.0
train: loss: 1112609.5 acc: 0.6904131174087524  val: loss: 1231159.0 acc: 0.8392201662063599
step: 49485 , time : 0.0010008811950683594
train: loss: 968705.9375 acc: 0.4563469886779785  val: loss: 1369689.75 acc: 0.8053243160247803
step: 49490 , time : 0.0
train: loss: 1512589.0 acc: 0.812563955783844  val: loss: 806789.1875 acc: 0.8607295751571655
step: 49495 , time : 0.0010004043579101562
train: loss: 1692593.125 acc: 0.4300018548965454  val: loss: 1419447.5 acc: 0.6910252571105957
step: 49500 , time : 0.0
train: loss: 1220212.25 acc: 0.42296934127807617  val: loss: 21

step: 49800 , time : 0.0
train: loss: 1180057.625 acc: 0.9582871198654175  val: loss: 408911.4375 acc: 0.7375682592391968
step: 49805 , time : 0.0010006427764892578
train: loss: 510618.34375 acc: 0.9838995933532715  val: loss: 407550.65625 acc: 0.8154562711715698
step: 49810 , time : 0.0
train: loss: 1231396.0 acc: 0.9590417742729187  val: loss: 826389.125 acc: 0.7122573852539062
step: 49815 , time : 0.0
train: loss: 681083.375 acc: 0.9666386246681213  val: loss: 1001246.0625 acc: 0.5767036080360413
step: 49820 , time : 0.0
train: loss: 347484.09375 acc: 0.9594876170158386  val: loss: 130578.1171875 acc: 0.9316458702087402
step: 49825 , time : 0.0010008811950683594
train: loss: 868559.0 acc: 0.8918445706367493  val: loss: 286434.65625 acc: 0.9429804086685181
step: 49830 , time : 0.0
train: loss: 952660.75 acc: 0.8274353742599487  val: loss: 426699.84375 acc: 0.9281381368637085
step: 49835 , time : 0.0
train: loss: 610580.4375 acc: 0.9235734939575195  val: loss: 464885.6875 acc: 0.93231